In [31]:
import pandas as pd
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

service=Service('C:/Users/Manych/chromium/chromedriver.exe')

In [11]:
def get_marks_models():
    '''
    возврашщает pd.Series в котором 
    индекс - название марки автомобиоя, 
    значения - списки названий моделей для каждой маркию
    
    '''
    url_for_marks_models = 'https://auto.ru/catalog/cars/all/'
    
    marks_models = pd.Series() 

    driver = webdriver.Chrome(service=service)  # открываем driver
    driver.maximize_window()                    # масксимизируем окно

    for pages_num in range(1,20):
        
        if pages_num==1: url = url_for_marks_models + '?view_type=list'
        else:            url = url_for_marks_models + '?page_num='+ str(pages_num) +'&view_type=list'
        
        driver.get(url)                         # открываем страницу по url
        res = driver.execute_script("return document.body.innerHTML;") # получаем html
        soup = BeautifulSoup(res, 'html.parser') # создаем обьект bs4.BeautifulSoup
        
        # Список html-ек перечня моделей каждой марки на странице:
        marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') 
    
        if not marks_on_page_list: 
            break
        
        for mark in marks_on_page_list:
            models_of_mark = mark.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
            link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
            mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
            mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
            mark_name = link_for_mark_name[mark_start:mark_end].upper() # получение обозначения марки  
        
            models_list = []
            for model in models_of_mark:
                link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
                model_start = mark_end+1                            # первый символ обозначения модели
                model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
                model_name = link_theme_auto[model_start:model_end].upper() # получение обозначения модели
                models_list.append(model_name)
                
            marks_models[mark_name] = models_list
        
        time.sleep(1) 
    # Закрываем процесс браузера:
    driver.quit()
    
    return marks_models

In [12]:
# model_url = 'https://auto.ru/moskva/cars/bmw/1er/used/'
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']

In [13]:
url_for_marks_models = 'https://auto.ru/catalog/cars/all/'

In [14]:
%%time
marks_models = get_marks_models()

<ipython-input-11-6cbf6382a67b>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  marks_models = pd.Series()


Wall time: 49.8 s


In [15]:
marks_models_for_parsing = marks_models.loc[marks_for_parsing]

# ----------------

In [26]:
marks_models_for_parsing

SKODA         [FABIA, FABIA_RS, FELICIA, FORMAN, KAROQ, KODI...
AUDI          [100, 200, 80, 90, A1, A2, A3, A4, A4_ALLROAD,...
HONDA         [ACCORD, ACTY, AIRWAVE, ASCOT, AVANCIER, BEAT,...
VOLVO         [164, 240_SERIES, 300_SERIES, 440, 460, 740, 7...
BMW           [1ER, 2ACTIVETOURER, 2ER, 2GRANDTOURER, 321, 3...
NISSAN        [100NX, 180SX, 200SX, 300ZX, 350Z, AD, ALMERA,...
INFINITI      [EX, FX, G35, JX, M, Q, Q30, Q50, Q60, Q70, QX...
MERCEDES      [190_SL, A_KLASSE, A_KLASSE_AMG, AMG_GLC_COUPE...
TOYOTA        [4RUNNER, ALPHARD, ALTEZZA, AQUA, ARISTO, AURI...
LEXUS         [CT, ES, GS, GS_F, GX, IS, LC, LS, LX, NX, RC,...
VOLKSWAGEN    [AMAROK, ARTEON, BEETLE, BORA, CADDY, CARAVELL...
MITSUBISHI    [3000_GT, AIRTREK, ASPIRE, ASX, CARISMA, CHARI...
dtype: object

In [28]:
marks_models_for_parsing[['BMW']]

BMW    [1ER, 2ACTIVETOURER, 2ER, 2GRANDTOURER, 321, 3...
dtype: object

In [35]:
# if 1:
def get_mark_convert(marks_models_for_parsing):
    '''
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной марки
    '''
    mark_convert = pd.Series() 
    
    
    driver = webdriver.Chrome(service=service)  # запускаем процесс браузера
    driver.maximize_window()                    # масксимизируем окно        

    for mark in marks_models_for_parsing[['BMW']].index:
#         print(mark)
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'  
            
            print(mark,model)
            driver.get(model_url) # открываем страницу по url
            models_list_res = driver.execute_script("return document.body.innerHTML;") # получаем htm
            
            # создаем обьект bs4.BeautifulSoup из html: 
            models_list_bs = BeautifulSoup(models_list_res, 'html.parser') 
    
            header_text = models_list_bs.find('h1').find_all('span')[0].text    
            model_human = header_text[header_text.find(mark)+len(mark)+1: header_text.find(' с пробегом')]   
            mark_convert[model] = model_human
            
    driver.quit()    # закрываем процесс браузера              
    return mark_convert

<ipython-input-35-a85f490cb386>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mark_convert = pd.Series()


In [37]:
mark_convert  = get_mark_convert(marks_models_for_parsing)

1ER                            1 серии
2ACTIVETOURER    2 серии Active Tourer
2ER                            2 серии
2GRANDTOURER       2 серии Gran Tourer
321                                321
340                                340
3ER                            3 серии
4                              4 серии
5ER                            5 серии
600                                600
6ER                            6 серии
7ER                            7 серии
8ER                            8 серии
I3                                  i3
I8                                  i8
IX                                  iX
M2                                  M2
M3                                  M3
M4                                  M4
M5                                  M5
M6                                  M6
M8                                  M8
X1                                  X1
X2                                  X2
X3                                  X3
X4                       

In [22]:
driver = webdriver.Chrome(service=service)  # открываем driver
driver.maximize_window()                    # масксимизируем окно

mark = 'BMW'

if 1:
#     driver.get(model_url+'?output_type=models_list') # открываем страницу по url
    driver.get('https://auto.ru/moskva/cars/bmw/3er/used/?output_type=models_list')
    
    models_list_res = driver.execute_script("return document.body.innerHTML;") # получаем html   
    
    # создаем обьект bs4.BeautifulSoup из html: 
    models_list_bs = BeautifulSoup(models_list_res, 'html.parser') 
    
    header_text = models_list_bs.find('h1').find_all('span')[0].text    
    model_human = header_text[header_text.find(mark)+len(mark)+1: header_text.find(' с пробегом')]    
    
    
driver.quit()    # закрываем процесс браузера    

In [23]:
model_human

'3 серии'

In [ ]:
<h1 class="ListingHead__title"><span>Купить BMW 1 серии с пробегом</span><span> В&nbsp;Москве</span></h1>

In [ ]:
def get_mark_convert_dict(marks_models_for_parsing):
    '''
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной марки
    '''
    driver = webdriver.Chrome(service=service)  # запускаем процесс браузера
    driver.maximize_window()                    # масксимизируем окно    
    
    model_generation_year = pd.DataFrame(columns = ['full_name','bodytype','generation_year'])
    
    for mark in marks_models_for_parsing.index:
        print(mark, end=' | ')
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
            tmp = get_generation_year(model_url)
            model_generation_year = model_generation_year.append(tmp)
            time.sleep(1)
            
    driver.quit()    # закрываем процесс браузера
    
    return model_generation_year

In [27]:
    # создаем обьект bs4.BeautifulSoup из html: 
    models_list_bs = BeautifulSoup(models_list_res, 'html.parser') 

    ttt = models_list_bs.find('h1').find_all('span')[0].text

In [33]:
ttt[ttt.find(mark)+len(mark)+1: ttt.find(' с пробегом')]

'1 серии'

In [ ]:
def get_generation_year(model_url):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название модели с указанием поколения, 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
        
    model_url - ссылка на страницу со списокм поколений модели
    
    '''
    driver.get(model_url+'?output_type=models_list') # открываем страницу по url
    
    models_list_res = driver.execute_script("return document.body.innerHTML;") # получаем html        
    
    # создаем обьект bs4.BeautifulSoup из html: 
    models_list_bs = BeautifulSoup(models_list_res, 'html.parser') 
    
    
    
    
    
#     # получаем список html-ек описаний для всех поколений модели  
#     posting_tag = models_list_bs.find_all('h3',class_ = "ListingItemGroup__title")
    
#     # формируем списки с указанием поколения модели, типом кузова поколения и годом начала выпуска поколения
#     # (необходимость указания типа кузова вызвана тем, что разные кузова зачастую переходят на следующее 
#     #  поколение в разные годы)
#     data = [
#     [tag.find('a',class_ = 'ListingItemTitle__link').text,             # поколение
#      tag.find('div',class_ = "ListingItemGroup__subtitle").text[:      # кузов
#                 tag.find('div',class_ = "ListingItemGroup__subtitle").text.find(' •')],
#      int(tag.find('div',class_ = "ListingItemGroup__subtitle").text    # год
#                 [tag.find('div',class_ = "ListingItemGroup__subtitle").text.find('(')+1:
#                  tag.find('div',class_ = "ListingItemGroup__subtitle").text.find('(')+5])] 
#         for tag in posting_tag]
    
#     result = pd.DataFrame(data,columns = ['full_name','bodytype','generation_year'])
    
    return result 

def get_model_generation_year(marks_models_for_parsing):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название марки и модели с указанием поколения 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
    
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной марки
    '''
    driver = webdriver.Chrome(service=service)  # запускаем процесс браузера
    driver.maximize_window()                    # масксимизируем окно    
    
    model_generation_year = pd.DataFrame(columns = ['full_name','bodytype','generation_year'])
    
    for mark in marks_models_for_parsing.index:
        print(mark, end=' | ')
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
            tmp = get_generation_year(model_url)
            model_generation_year = model_generation_year.append(tmp)
            time.sleep(1)
            
    driver.quit()    # закрываем процесс браузера
    
    return model_generation_year

In [3]:
model_generation_year_all = pd.read_csv('./Project_7_data/model_generation_year_all.csv')